In [ ]:
# script to set up many simulations
# import pandas as pd
# import pickle
import numpy as np
import cantera as ct
import concurrent.futures
import copy
import rmgpy.chemkin
import rmgpy.species
import rmgpy.tools.canteramodel
import rmgpy.constants

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
def plot_kinetics(rxns, labels=None):
    """Function for plotting reaction kinetics
    Takes in a list of RMG reactions (rmgpy.reaction.Reaction) or a single reaction
    """
    plt.xlabel('1000 / T (K^-1)')
    plt.ylabel('log10(k)')

    if type(rxns) != list:
        rxns = [rxns]

    T = np.linspace(300, 3000, 1001)
    for rxn in rxns:
        k = np.zeros(len(T))
        for i in range(0, len(T)):
            k[i] = rxn.get_rate_coefficient(T[i], 1013250)
        plt.plot(1000.0 / T, np.log10(k))

    if labels:
        plt.legend(labels)
#     plt.show()

In [ ]:
# load in the rmg mechanism
chemkin_file = '/home/moon/autoscience/fuels/with_lib/butane_20240413/chem_annotated.inp'
dict_file = '/home/moon/autoscience/fuels/with_lib/butane_20240413/species_dictionary.txt'


chemkin_file = '/home/moon/autoscience/fuels/butane_20240501/chem_annotated.inp'
dict_file = '/home/moon/autoscience/fuels/butane_20240501/species_dictionary.txt'


chemkin_file = '/home/moon/autoscience/aramco/total_aramco_mech.inp'
dict_file = '/home/moon/autoscience/aramco/species_dictionary.txt'

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)


In [ ]:
def perturb_reaction(rxn, delta):  # 0.1 is default
    # takes in an RMG reaction object
    # delta is the ln(k) amount to perturb the A factor
    # delta is a multiplicative factor- units don't matter, yay!
    # does not deepycopy because there's some issues with rmgpy.reactions copying
    if type(rxn.kinetics) == rmgpy.kinetics.chebyshev.Chebyshev:
        rxn.kinetics.coeffs.value_si[0][0] += np.log10(1.0 + delta)
    elif type(rxn.kinetics) in [rmgpy.kinetics.falloff.Troe, rmgpy.kinetics.falloff.ThirdBody, rmgpy.kinetics.falloff.Lindemann]:
        if hasattr(rxn.kinetics, 'arrheniusHigh'):
            rxn.kinetics.arrheniusHigh.A.value *= np.exp(delta)
        if hasattr(rxn.kinetics, 'arrheniusLow'):
            rxn.kinetics.arrheniusLow.A.value *= np.exp(delta)
    elif type(rxn.kinetics) == rmgpy.kinetics.arrhenius.MultiArrhenius:
        for j in range(len(rxn.kinetics.arrhenius)):
            rxn.kinetics.arrhenius[j].A.value *= np.exp(delta)
    elif type(rxn.kinetics) == rmgpy.kinetics.arrhenius.PDepArrhenius:
        for j in range(len(rxn.kinetics.arrhenius)):
            if type(rxn.kinetics.arrhenius[j]) == rmgpy.kinetics.arrhenius.Arrhenius:
                rxn.kinetics.arrhenius[j].A.value *= np.exp(delta)
            else:
                raise ValueError(f'weird kinetics {str(rxn.kinetics)}')
    elif type(rxn.kinetics) == rmgpy.kinetics.arrhenius.MultiPDepArrhenius:
        for i in range(len(rxn.kinetics.arrhenius)):
            for j in range(len(rxn.kinetics.arrhenius[i].arrhenius)):
                if type(rxn.kinetics.arrhenius[i].arrhenius[j]) == rmgpy.kinetics.arrhenius.Arrhenius:
                    rxn.kinetics.arrhenius[i].arrhenius[j].A.value *= np.exp(delta)
                elif type(rxn.kinetics.arrhenius[i].arrhenius[j]) == rmgpy.kinetics.arrhenius.MultiArrhenius:
                    for k in range(len(rxn.kinetics.arrhenius[i].arrhenius[j].arrhenius)):
                        rxn.kinetics.arrhenius[i].arrhenius[j].arrhenius[k].A.value *= np.exp(delta)
                else:
                    raise ValueError(f'weird kinetics {str(rxn.kinetics)}')

    else:  # Arrhenius
        rxn.kinetics.A.value *= np.exp(delta)


# Regular Arrhenius Demo

In [ ]:
# load a fresh model
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)

T = 1000
P = 1013250
delta = 0.1
N = 5
count = 0
fig, ax = plt.subplots(N)
fig.set_figheight(30)
for i in range(len(reaction_list)):
    if count >= N:
        break
    if type(reaction_list[i].kinetics) == rmgpy.kinetics.arrhenius.Arrhenius:
        ref_kinetics = copy.deepcopy(reaction_list[i].kinetics)
        plt.sca(ax[count])
        perturb_reaction(reaction_list[i], delta)
        plot_kinetics([ref_kinetics, reaction_list[i].kinetics], labels=['original', 'perturbed'])
        count += 1

# Chebyshev demo

In [ ]:
# load a fresh model
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)

T = 1000
P = 1013250
delta = 0.1
N = 5
count = 0
fig, ax = plt.subplots(N)
fig.set_figheight(30)
for i in range(len(reaction_list)):
    if count >= N:
        break
    if type(reaction_list[i].kinetics) == rmgpy.kinetics.chebyshev.Chebyshev:
        ref_kinetics = copy.deepcopy(reaction_list[i].kinetics)
        plt.sca(ax[count])
        perturb_reaction(reaction_list[i], delta)
        plot_kinetics([ref_kinetics, reaction_list[i].kinetics], labels=['original', 'perturbed'])
        count += 1

# Multi-arrhenius

In [ ]:
# load a fresh model
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)

T = 1000
P = 1013250
delta = 0.1
N = 5
count = 0
fig, ax = plt.subplots(N)
fig.set_figheight(30)
for i in range(len(reaction_list)):
    if count >= N:
        break
    if type(reaction_list[i].kinetics) == rmgpy.kinetics.arrhenius.MultiArrhenius:
        ref_kinetics = copy.deepcopy(reaction_list[i].kinetics)
        plt.sca(ax[count])
        perturb_reaction(reaction_list[i], delta)
        plot_kinetics([ref_kinetics, reaction_list[i].kinetics], labels=['original', 'perturbed'])
        count += 1

# Multi PDEP

In [ ]:
# load a fresh model
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)

T = 1000
P = 1013250
delta = 0.1
N = 5
count = 0
fig, ax = plt.subplots(N)
fig.set_figheight(30)
for i in range(len(reaction_list)):
    if count >= N:
        break
    if type(reaction_list[i].kinetics) == rmgpy.kinetics.arrhenius.MultiPDepArrhenius:
        ref_kinetics = copy.deepcopy(reaction_list[i].kinetics)
        plt.sca(ax[count])
        perturb_reaction(reaction_list[i], delta)
        plot_kinetics([ref_kinetics, reaction_list[i].kinetics], labels=['original', 'perturbed'])
        count += 1

# PDEP Arrhenius

In [ ]:
# load a fresh model
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)

T = 1000
P = 1013250
delta = 0.1
N = 5
count = 0
fig, ax = plt.subplots(N)
fig.set_figheight(30)
for i in range(len(reaction_list)):
    if count >= N:
        break
    if type(reaction_list[i].kinetics) == rmgpy.kinetics.arrhenius.PDepArrhenius:
        ref_kinetics = copy.deepcopy(reaction_list[i].kinetics)
        plt.sca(ax[count])
        perturb_reaction(reaction_list[i], delta)
        plot_kinetics([ref_kinetics, reaction_list[i].kinetics], labels=['original', 'perturbed'])
        count += 1

# Troe

In [ ]:
# load a fresh model
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)

T = 1000
P = 1013250
delta = 0.1
N = 5
count = 0
fig, ax = plt.subplots(N)
fig.set_figheight(30)
for i in range(len(reaction_list)):
    if count >= N:
        break
    if type(reaction_list[i].kinetics) == rmgpy.kinetics.falloff.Troe:
        ref_kinetics = copy.deepcopy(reaction_list[i].kinetics)
        plt.sca(ax[count])
        perturb_reaction(reaction_list[i], delta)
        plot_kinetics([ref_kinetics, reaction_list[i].kinetics], labels=['original', 'perturbed'])
        count += 1



# Third-Body

In [ ]:
# load a fresh model
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)

T = 1000
P = 1013250
delta = 0.1
N = 5
count = 0
fig, ax = plt.subplots(N)
fig.set_figheight(30)
for i in range(len(reaction_list)):
    if count >= N:
        break
    if type(reaction_list[i].kinetics) == rmgpy.kinetics.falloff.ThirdBody:
        ref_kinetics = copy.deepcopy(reaction_list[i].kinetics)
        plt.sca(ax[count])
        perturb_reaction(reaction_list[i], delta)
        plot_kinetics([ref_kinetics, reaction_list[i].kinetics], labels=['original', 'perturbed'])
        count += 1



# Lindemann

In [ ]:
# load a fresh model
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_file, dictionary_path=dict_file)

T = 1000
P = 1013250
delta = 0.1
N = 5
count = 0
fig, ax = plt.subplots(N)
fig.set_figheight(30)
for i in range(len(reaction_list)):
    if count >= N:
        break
    if type(reaction_list[i].kinetics) == rmgpy.kinetics.falloff.Lindemann:
        print(i)
        ref_kinetics = copy.deepcopy(reaction_list[i].kinetics)
        plt.sca(ax[count])
        perturb_reaction(reaction_list[i], delta)
        plot_kinetics([ref_kinetics, reaction_list[i].kinetics], labels=['original', 'perturbed'])
        count += 1





In [ ]:
delta = 0.1
for reaction in reaction_list:
    try:
        perturb_reaction(reaction, delta)
    except AttributeError:
        print('skipping ', type(reaction.kinetics))

